# 1: The Fermi-LAT data

## Introduction

Hello *Fermi* initiate! 

In these tutorials you will be taken through your first *Fermi*-LAT analysis. 

We will look at:

* What *Fermi* data looks like.
* What the tools are, and what they do.
* The Python interface to the tools.
* Using a python package called Fermipy which will automate a lot of this for you.
* Some more advanced point source analysis techniques.
* Some other useful sources of information passed down through many generations of PhD students here in the group.
* Some common (and highly irritating) errors with the *Fermi* software, and how to solve them.

### Acknowledgements

These tutorials are very much based on an earlier set of tutorials written by some of our former graduate students for graduate students. In some places this is almost word for word identical (as users of Linux know, if something works, don't change it!). In addition, the absolutely incredible *Fermi* team at NASA and the University of Delaware (Liz, Julie, Jamie Holder, Jeremy), and also the work of Durham alumni Dr Tom Armstrong,  Dr Jamie Graham and Dr Max Harvey. 

"*Dale a tu cuerpo alegría Macarena,*

*Que tu cuerpo es pa' darle alegría why cosa buena,*

*Dale a tu cuerpo alegría, Macarena,*

*Hey Macarena* "

### How do we get the *Fermi* data?

The LAT data is stored at https://fermi.gsfc.nasa.gov/cgi-bin/ssc/LAT/LATDataQuery.cgi, and the page looks like this. 

![Data Server](../img/LATData.png)

We're going to be looking at a particularly interesting source of $\gamma$-rays, a reasonably bright and highly variable radio galaxy called NGC 1275 (studied in depth by Dr Anthony Brown and Dr Cameron Rulten here in the $\gamma$-ray astronomy group at Durham). You can read Anthony's paper on [arXiv](https://arxiv.org/abs/1101.2687), but it isn't essential reading for this tutorial, just a good example of a *Fermi* paper.

Firstly, lets check we're located in the correct directory for this tutorial. For tutorial 1 to work properly we need to be inside the ```tutorial-1-fermidata``` directory.

In [ ]:
current_dir = %pwd
print(current_dir)

The query made to the data server in order to get the data used for this tutorial is saved in a file called ```data_query.dat```. Lets inspect this file by printing its contents:

In [ ]:
!cat data_query.dat

We can see we have to make a few choices in order to get our data from the server. Lets take a look at what we chose, and why.

### Coordinates
The equatorial coordinates simply refer to the right ascension (RA) and declination (DEC) of our target, NGC 1275 If you're unsure on RA and DEC... [you can find some background here](https://starwalk.space/en/news/celestial-coordinates). Note our RA is in degrees (it's simpler this way, trust us!).

You can also use Galactic coordinates for this choice, which is particularly convenient if you're looking at $\gamma$-ray sources on the Galactic plane.

### Time Range
*Fermi* launched in 2008, and has spent most of its time observing the whole sky, hence there is now about 16 years of public data available. Generally speaking, the more data we have in our analysis, the more photons we have, and often the better our results will be. That said, with variable sources (such as NGC 1275), source properties change over time, and we must investigate how this happens, but more on that later. For our study here, we have 5 months of data, to save on file space and computation time. You will find that a decade long analysis can take quite a while. The current record for a single analysis is 3 days (at Durham).

We define time with *Fermi* in 3 ways. We have the Gregorian calendar, which is the calendar we use on a day to day basis. We also have the Modified Julian Calendar, which counts days in MJD. Finally, we have Mission Elapsed Time (MET). MET is expressed in seconds since a particular date in 2001, and is the preferred way to express *Fermi* time.

You can use [NASA's online HEASARC xTime](https://heasarc.gsfc.nasa.gov/cgi-bin/Tools/xTime/xTime.pl) tool to convert between dates/times, but in general we should use MET.

When we look at the time range in our Query, we see it runs from 2010-2011, even though we input it in MET. The data query always comes out in Gregorian dates, as well as MET.

### Energy Range

*Fermi* has an effective energy range of 100 MeV to 500 GeV, which is generally referred to as the 'high energy' $\gamma$-ray band. We say it's an effective energy range, because it does detect photons down to 30 MeV, and up to approximately 1 TeV, although these higher energy photons are extremely rare for a detector the size of the LAT. 

As a result, we tend to use this 100 MeV to 500 GeV range, this allows us to explore the spectrum of a source to its fullest.

### Search Radius

Our search radius corresponds to the size of the area we want to look at.

We use a larger (20+ degrees) region of interest (ROI) for:

- bright sources near your ROI. This can allow flux from a source outside the ROI to work its way into your model. We'll explore this in detail when we come to likelihood fitting.
- working on (or close to) the galactic plane. There are lots of bright sources near here, and the best fit of the interstellar emission model will come from a large ROI.
- working with sources that peak towards lower energies. The point spread function (PSF) of the *Fermi-LAT* is **very** energy dependent. Its 68% containment region goes from several degrees at 100 MeV to 0.1 degrees at energies > 1 GeV. This can be rather frustrating to deal with, and is the principle reason we use likelihood fitting techniques.

And a smaller (10 - 20 degrees) ROI if

- you're working on an AGN off the plane, without anything bright nearby.
- you're working at high energies (>1 GeV). The *Fermi* PSF allows you to cut down your ROI in this case.


### What does our data look like?

If we take a look in the data directory, there are a few files:


In [ ]:
#If running this notebook directly in the Durham tutorial docker image
!ls /data/

At first sight, the names of these files are a bit incomprehensible. The file format is FITS, which stands for 'Flexible Image Transfer System', and is the standard way to store astronomical data. The files with 'PHXX' at the end (where XX = 00 for example), are our photon files, and these store the information about all the $\gamma$-rays detected within our time range, energy range and region of interest. The one with 'SC00' at the end is our spacecraft file, and this contains information about where the LAT was pointed as a function of time. 

Lets open up a photon file and the spacecraft file and take a look inside:

In [ ]:
from astropy.io import fits as fits

PH_file = fits.open('/data/L1910170847288E15477E17_PH00.fits')

SC_file = fits.open('/data/L1910170847288E15477E17_SC00.fits')

If we print the header of the photon file, we can see all of our data query, plus some more information.

In [ ]:
PH_file[0].header

Everything looks good, apart from our time range! Although we have requested a relatively small amount of data (not even half a year!), the *Fermi* data file server split this into files of different, smaller, and more manageable sizes. The reason for choosing 5 months of data is also influenced by the fact that GitHub (where this tutorial and data is stored) has a file size limit. PH00 gives the photons from roughly the first three months of observations, which is correct. Another curiosity is that of our telescope name: GLAST. GLAST stands for Gamma-ray Large Area Space Telescope, which is what *Fermi* was called during its design and construction. There are references to this previous name throughout the software and data: the *Fermi* tools all begin with 'gt' which stands for GLAST tools. 

A FITS file will contain several "HDUs", each of which contain different information. Here we looked at HDU 0. Traditionally, this is the HDU that contains image data. However in gamma ray astronomy, we tend to count events - not look at images. So if we try to access this HDU:

In [ ]:
print(PH_file[0].data)

There is nothing there. Instead all of the information is kept in HDU 1.

In [ ]:
print(PH_file[1].data[0])

This is just the first photon in the file. As you can see it contains a lot of information. We have no idea what all of this means currently, so let's look at the header of this HDU:

In [ ]:
PH_file[1].header

So we have a load of information. If you look at the columns labelled "TTYPE[x]", these tell you what information is kept for each of the photons. So if we want to look at the energy, RA, and Dec of the first photon, we would do the following:

In [ ]:
Energy = PH_file[1].data[0]['Energy']
RA = PH_file[1].data[0]['RA']
DEC = PH_file[1].data[0]['DEC']
print(f"Energy: {Energy} [MeV]   RA,DEC: {RA}, {DEC} [degrees]")

Or alternatively:

In [ ]:
PH_file[1].data[0][0]

We can now take a quick look at the header of the spacecraft file too:

In [ ]:
SC_file[0].header

And we get a long list of information regarding the spacecraft, fun! It is important to have a basic understanding of the data which we are operating on, as when things go wrong, or something unexpected arises, this understanding helps us troubleshoot. 

Feel free to play about with these files as much as you want. For example look at what information is contained in the PH file, or explore the SC file in further depth. However, we'll leave that to your own curiosity. In the next tutorial, we'll move onto discussion of the *Fermi* Tools.